<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/files_2020/cce-color.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Validación de Datos de Bodega
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Septiembre 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Descarga de datos y unificación de fuentes**|
|**Descripción y alcance**|Notebook para la descarga y unificación de los datos que alimentan a la base unificada.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos - GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Maria del Pilar Suarez Sebastian</td></tr> <tr><td>*Cargo:*</td> <td>Subdirectora Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

In [1]:
### Paquetes usados para la validación de datos

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import unidecode
import libreria.data_management as dm
# from dataprep.clean import clean_duplication
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.0f}'.format

from libreria import login # Lógica para el inicio de sesión a las fuentes de datos

# Cargue de las conexiones a las fuentes de datos
cnxn_sandbox, cursor_sandbox = login.login_sql(login.tipo_bodega.SANDBOX)
cnxn_reports, cursor_reports = login.login_sql(login.tipo_bodega.REPORTS)

In [2]:
db_maestra_B = pd.read_csv('../../../muestras de datos/procesados/bronce/SECOP_ENERO23.csv', sep=';')
db_maestra_S = pd.read_csv('../../../muestras de datos/procesados/silver/SECOP_ENERO23.csv', sep=';')

In [3]:
year = '2023'
months = '1' # 1: Enero, 2: Febrero, 3: Marzo, etc.


SII_B = db_maestra_B[db_maestra_B['Fuente'] == 'SECOP II']
SII_B['Fecha_firma'] = pd.to_datetime(SII_B['Fecha_firma'])
filtered_b = SII_B[~SII_B['Estado'].isin(['Borrador', 'Cancelado', 'En aprobación', 'enviado Proveedor'])]

SII_S = db_maestra_S[db_maestra_S['Fuente'] == 'SECOP II']
SII_S['Fecha_firma'] = pd.to_datetime(SII_S['Fecha_firma'])
filtered_s = SII_S[~SII_S['Estado'].isin(['Borrador', 'Cancelado', 'En aprobación', 'enviado Proveedor'])]

## Test 1

In [4]:
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023 ----
## El calculo se realiza para optener los contratos unicos tanto en valor como en cantidad.       
## Los datos corresponden con la información reportada por las entidades estatales en el SECOP II 
# Datos extraidos desde la bodega
test1 = pd.read_sql_query(f"SELECT DISTINCT MONTH(VALOR.AprovalDate) AS 'MES', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['ID_Contrato', 'Fecha_firma'])['Valor_contrato'].max().reset_index()
grouped_b['MES'] = grouped_b['Fecha_firma'].dt.month
result_b = grouped_b.groupby('MES').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
result_b.columns = ['MES(B)', 'CONTRATOS(B)', 'VALOR(B)']

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['ID_Contrato', 'Fecha_firma'])['Valor_contrato'].max().reset_index()
grouped_s['MES'] = grouped_s['Fecha_firma'].dt.month
result_s = grouped_s.groupby('MES').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
result_s.columns = ['MES(S)', 'CONTRATOS(S)', 'VALOR(S)']

# Ordena el resultado por MES
result_b = result_b.sort_values(by='MES(B)', ascending=True)
result_s = result_s.sort_values(by='MES(S)', ascending=True)

test1[['MES(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test1[['MES(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    RT1DC = int(test1['CONTRATOS']) == int(test1['CONTRATOS(B)'])
    RT1DV = int(test1['VALOR'])        == int(test1['VALOR(B)'])
except:
    RT1DC = False
    RT1DV = False
    print('Error al validar el proceso de descarga')

try:
    RT1LC = int(test1['CONTRATOS(B)']) == int(test1['CONTRATOS(S)'])
    RT1LV = int(test1['VALOR(B)'])     == int(test1['VALOR(S)'])
except:
    RT1LC = False
    RT1LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT1DC)
print('Valor: ', RT1DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT1LC)
print('Valor: ', RT1LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 2

In [5]:
# Datos extraidos desde la bodega
test2 = pd.read_sql_query(f"SELECT COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['ID_Contrato', 'Fecha_firma']).agg({'Valor_contrato': 'max'}).reset_index()
contratos_b = len(grouped_b['ID_Contrato'].unique())
valor_b = grouped_b['Valor_contrato'].sum()

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['ID_Contrato', 'Fecha_firma']).agg({'Valor_contrato': 'max'}).reset_index()
contratos_s= len(grouped_s['ID_Contrato'].unique())
valor_s = grouped_s['Valor_contrato'].sum()

test2[['CONTRATOS(B)', 'VALOR(B)', 'CONTRATOS(S)', 'VALOR(S)']] = contratos_b, valor_b, contratos_s, valor_s

try:
    RT2DC = int(test2['CONTRATOS'])    == int(test2['CONTRATOS(B)'])
    RT2DV = int(test2['VALOR'])        == int(test2['VALOR(B)'])
except:
    RT2DC = False
    RT2DV = False
    print('Error al validar el proceso de descarga')

try:
    RT2LC = int(test2['CONTRATOS(B)']) == int(test2['CONTRATOS(S)'])
    RT2LV = int(test2['VALOR(B)'])     == int(test2['VALOR(S)'])
except:
    RT2LC = False
    RT2LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT2DC)
print('Valor: ',     RT2DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT2LC)
print('Valor: ',     RT2LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 3

In [6]:
# CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test3 = pd.read_sql_query(f"SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
result_b = filtered_b.groupby(['Modalidad', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
result_b = result_b.groupby('Modalidad').agg({'ID_Contrato': 'count','Valor_contrato': 'sum'}).reset_index()
result_b.columns = ['MODALIDAD', 'CONTRATOS', 'VALOR']
result_b = result_b.sort_values(by='MODALIDAD', ascending=True)

# Traducción de la consulta (Base silver)
result_s = filtered_s.groupby(['Modalidad', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
result_s = result_s.groupby('Modalidad').agg({'ID_Contrato': 'count','Valor_contrato': 'sum'}).reset_index()
result_s.columns = ['MODALIDAD', 'CONTRATOS', 'VALOR']
result_s = result_s.sort_values(by='MODALIDAD', ascending=True)

test3[['MODALIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test3[['MODALIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    test3['RT3DC'] = test3['CONTRATOS'] == test3['CONTRATOS(B)']
    RT3DC = test3[~test3['CONTRATOS'] == 'None'][test3['RT3DC'] == False].shape[0] == 0
    test3['RT3DV'] = test3['VALOR'] == test3['VALOR(B)']
    RT3DV = test3[~test3['CONTRATOS'] == 'None'][test3['RT3DV'] == False].shape[0] == 0
except:
    RT3DC = False
    RT3DV = False
    print('Error al validar el proceso de descarga')

try:
    test3['RT3LC'] = test3['CONTRATOS(B)'] == test3['CONTRATOS(S)']
    RT3LC = test3[~test3['CONTRATOS(B)'].isna()][test3['RT3LC'] == False].shape[0] == 0
    test3['RT3LV'] = test3['VALOR(B)'] == test3['VALOR(S)']
    RT3LV = test3[~test3['VALOR(B)'].isna()][test3['RT3LV'] == False].shape[0] == 0
except:
    RT3LC = False
    RT3LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT3DC)
print('Valor: ',     RT3DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT3LC)
print('Valor: ',     RT3LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  False
Valor:  False


## Test 4

In [7]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test4 = pd.read_sql_query(f"SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE	YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['Tipo_de_contrato', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
result_b = grouped_b.groupby('Tipo_de_contrato').agg({'ID_Contrato': 'nunique','Valor_contrato': 'sum'}).reset_index()
result_b.rename(columns={'Tipo_de_contrato': 'MODALIDAD', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'}, inplace=True)
result_b.sort_values(by='MODALIDAD', ascending=True, inplace=True)

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['Tipo_de_contrato', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
result_s = grouped_s.groupby('Tipo_de_contrato').agg({'ID_Contrato': 'nunique','Valor_contrato': 'sum'}).reset_index()
result_s.rename(columns={'Tipo_de_contrato': 'MODALIDAD', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'}, inplace=True)
result_s.sort_values(by='MODALIDAD', ascending=True, inplace=True)

test4[['MODALIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test4[['MODALIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    test4['RT4DC'] = test4['CONTRATOS'] == test4['CONTRATOS(B)']
    RT4DC = test4[test4['RT4DC'] == False].shape[0] == 0
    test4['RT4DV'] = test4['VALOR'] == test4['VALOR(B)']
    RT4DV = test4[test4['RT4DV'] == False].shape[0] == 0
except:
    RT4DC = False
    RT4DV = False
    print('Error al validar el proceso de descarga')

try:
    test4['RT4LC'] = test4['CONTRATOS(B)'] == test4['CONTRATOS(S)']
    RT4LC = test4[test4['RT4LC'] == False].shape[0] == 0
    test4['RT4LV'] = test4['VALOR(B)'] == test4['VALOR(S)']
    RT4LV = test4[test4['RT4LV'] == False].shape[0] == 0
except:
    RT4LC = False
    RT4LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT4DC)
print('Valor: ',     RT4DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT4LC)
print('Valor: ',     RT4LV)

Etapa de descarga:
Contratos:  True
Valor:  False
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 5

In [8]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test5 = pd.read_sql_query(f"SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['ID_Contrato', 'Nombre_Entidad']).agg({'Valor_contrato': 'max'}).reset_index()
result_b = grouped_b.groupby('Nombre_Entidad').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
result_b.columns = ['ENTIDAD', 'CONTRATOS', 'VALOR']
result_b = result_b.sort_values(by='VALOR', ascending=False).head(20)
result_b = result_b.reset_index()
result_b = result_b.drop(columns=['index'])

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['ID_Contrato', 'Nombre_Entidad']).agg({'Valor_contrato': 'max'}).reset_index()
result_s = grouped_s.groupby('Nombre_Entidad').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
result_s.columns = ['ENTIDAD', 'CONTRATOS', 'VALOR']
result_s = result_s.sort_values(by='VALOR', ascending=False).head(20)
result_s = result_s.reset_index()
result_s = result_s.drop(columns=['index'])

test5[['ENTIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test5[['ENTIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    test5['RT5DC'] = test5['CONTRATOS'] == test5['CONTRATOS(B)']
    RT5DC = test5[test5['RT5DC'] == False].shape[0] == 0
    test5['RT5DV'] = test5['VALOR'] == test5['VALOR(B)']
    RT5DV = test5[test5['RT5DV'] == False].shape[0] == 0
except:
    RT5DC = False
    RT5DV = False
    print('Error al validar el proceso de descarga')

try:
    test5['RT5LC'] = test5['CONTRATOS(S)'] == test5['CONTRATOS(B)']
    RT5LC = test5[test5['RT5LC'] == False].shape[0] == 0
    test5['RT5LV'] = test5['VALOR(S)'] == test5['VALOR(B)']
    RT5LV = test5[test5['RT5LV'] == False].shape[0] == 0
except:
    RT5LC = False
    RT5LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT5DC)
print('Valor: ',     RT5DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT5LC)
print('Valor: ',     RT5LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 6

In [9]:
## DATOS DE LA ENTIDAD (Secretaría Distrital de Integración Social), CODIGO 702271321
## CANTIDAD DE CONTRATOS Y VALOR POR MES DEL AÑO 2023
# Datos extraidos desde la bodega
test6 = pd.read_sql_query(f"SELECT DISTINCT MONTH(VALOR.AprovalDate)	AS 'MES', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS VALOR FROM (SELECT DISTINCT V.ContractReference, V.AprovalDate, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor') AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.AprovalDate) VALOR GROUP BY MONTH(VALOR.AprovalDate) ORDER BY MONTH(VALOR.AprovalDate) ASC", cnxn_reports)

filtered_b = filtered_b[filtered_b['ID_Entidad'] == 702271321]
filtered_s = filtered_s[filtered_s['ID_Entidad'] == 702271321]

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['ID_Contrato', 'Fecha_firma'])['Valor_contrato'].max().reset_index()
result_b = filtered_b.groupby(filtered_b['Fecha_firma'].dt.month).agg({'ID_Contrato': 'nunique', 'Valor_contrato': 'sum'}).reset_index()
result_b.rename(columns={'Fecha_firma': 'MES', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'}, inplace=True)
result_b.sort_values(by='MES', ascending=True, inplace=True)

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['ID_Contrato', 'Fecha_firma'])['Valor_contrato'].max().reset_index()
result_s = filtered_s.groupby(filtered_b['Fecha_firma'].dt.month).agg({'ID_Contrato': 'nunique', 'Valor_contrato': 'sum'}).reset_index()
result_s.rename(columns={'Fecha_firma': 'MES', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'}, inplace=True)
result_s.sort_values(by='MES', ascending=True, inplace=True)

test6[['MES(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test6[['MES(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    RT6DC = int(test6['CONTRATOS'])    == int(test6['CONTRATOS(B)'])
    RT6DV = int(test6['VALOR'])        == int(test6['VALOR(B)'])
except:
    RT6DC = False
    RT6DV = False
    print('Error al validar el proceso de descarga')

try:
    RT6LC = int(test6['CONTRATOS(B)']) == int(test6['CONTRATOS(S)'])
    RT6LV = int(test6['VALOR(B)'])     == int(test6['VALOR(S)'])
except:
    RT6LC = False
    RT6LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT6DC)
print('Valor: ',     RT6DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT6LC)
print('Valor: ',     RT6LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 7

In [10]:
## CANTIDAD DE CONTRATOS Y VALOR POR MODALIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test7 = pd.read_sql_query(f"SELECT DISTINCT VALOR.ProcedureProfileLabel AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.ProcedureProfileLabel, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.ProcedureProfileLabel) VALOR GROUP BY VALOR.ProcedureProfileLabel ORDER BY VALOR.ProcedureProfileLabel ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['ID_Contrato', 'Modalidad']).agg({'Valor_contrato': 'max'}).reset_index()
final_b = grouped_b.groupby('Modalidad').agg({'ID_Contrato': 'nunique', 'Valor_contrato': 'sum'}).reset_index()
final_b.columns = ['MODALIDAD', 'CONTRATOS', 'VALOR']
final_b.sort_values(by='MODALIDAD', ascending=True, inplace=True)

# Traducción de la consulta (Base silver)
grouped_s = filtered_s.groupby(['ID_Contrato', 'Modalidad']).agg({'Valor_contrato': 'max'}).reset_index()
final_s = grouped_s.groupby('Modalidad').agg({'ID_Contrato': 'nunique', 'Valor_contrato': 'sum'}).reset_index()
final_s.columns = ['MODALIDAD', 'CONTRATOS', 'VALOR']
final_s.sort_values(by='MODALIDAD', ascending=True, inplace=True)

test7[['MODALIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = final_b
test7[['MODALIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = final_s

try:
    test7['RT7DC'] = test7['CONTRATOS'] == test7['CONTRATOS(B)']
    RT7DC = test7[test7['RT7DC'] == False].shape[0] == 0
    test7['RT7DV'] = test7['VALOR'] == test7['VALOR(B)']
    RT7DV = test7[test7['RT7DV'] == False].shape[0] == 0
except:
    RT7DC = False
    RT7DV = False
    print('Error al validar el proceso de descarga')

try:
    test7['RT7LC'] = test7['CONTRATOS(S)'] == test7['CONTRATOS(B)']
    RT7LC = test7[~test7['CONTRATOS(B)'].isna() & (test7['RT7LC'] == False)].shape[0] == 0
    test7['RT7LV'] = test7['VALOR(S)'] == test7['VALOR(B)']
    RT7LV = test7[~test7['VALOR(B)'].isna() & (test7['RT7LV'] == False)].shape[0] == 0
except:
    RT7LC = False
    RT7LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT7DC)
print('Valor: ',     RT7DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT7LC)
print('Valor: ',     RT7LV)

Etapa de descarga:
Contratos:  False
Valor:  False
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 8

In [11]:
## CANTIDAD DE CONTRATOS Y VALOR POR TIPO DE CONTRATO DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test8 = pd.read_sql_query(f"SELECT DISTINCT VALOR.TypeContract AS 'MODALIDAD', COUNT(DISTINCT VALOR.ContractReference) AS 'CONTRATOS', SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.TypeContract, MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.TypeContract) VALOR GROUP BY VALOR.TypeContract ORDER BY VALOR.TypeContract ASC", cnxn_reports)

# Traducción de la consulta (Base bronce)
grouped_b = filtered_b.groupby(['Tipo_de_contrato', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
agg_b = grouped_b.groupby('Tipo_de_contrato').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
agg_b = agg_b.rename(columns={'Tipo_de_contrato': 'MODALIDAD', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'})
agg_b = agg_b.sort_values(by='MODALIDAD', ascending=True)

# Traducción de la consulta (Base silver)
grouped_s = filtered_b.groupby(['Tipo_de_contrato', 'ID_Contrato']).agg({'Valor_contrato': 'max'}).reset_index()
agg_s = grouped_s.groupby('Tipo_de_contrato').agg({'ID_Contrato': 'count', 'Valor_contrato': 'sum'}).reset_index()
agg_s = agg_s.rename(columns={'Tipo_de_contrato': 'MODALIDAD', 'ID_Contrato': 'CONTRATOS', 'Valor_contrato': 'VALOR'})
agg_s = agg_s.sort_values(by='MODALIDAD', ascending=True)

test8[['MODADLIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = agg_b
test8[['MODADLIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = agg_s

try:
    test8['RT8DC'] = test8['CONTRATOS'] == test8['CONTRATOS(B)']
    RT8DC = test8[test8['RT8DC'] == False].shape[0] == 0
    test8['RT8DV'] = test8['VALOR'] == test8['VALOR(B)']
    RT8DV = test8[test8['RT8DV'] == False].shape[0] == 0
except:
    RT8DC = False
    RT8DV = False
    print('Error al validar el proceso de descarga')

try:
    test8['RT8LC'] = test8['CONTRATOS(S)'] == test8['CONTRATOS(B)']
    RT8LC = test8[test8['RT8LC'] == False].shape[0] == 0
    test8['RT8LV'] = test8['VALOR(S)'] == test8['VALOR(B)']
    RT8LV = test8[test8['RT8LV'] == False].shape[0] == 0
except:
    RT8LC = False
    RT8LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT8DC)
print('Valor: ',     RT8DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT8LC)
print('Valor: ',     RT8LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Test 9

In [12]:
## TOP 20 - CANTIDAD DE CONTRATOS Y VALOR POR ENTIDAD DE LOS MESES DEL 1 AL 6 DEL AÑO 2023
# Datos extraidos desde la bodega
test9 = pd.read_sql_query(f"SELECT DISTINCT TOP 20 VALOR.[Nombre Entidad] AS 'ENTIDAD', COUNT(DISTINCT VALOR.ContractReference)	AS 'CONTRATOS',	SUM(VALOR.VALOR) AS 'VALOR' FROM (SELECT DISTINCT V.ContractReference, V.[Nombre Entidad], MAX(V.ContractValue) AS VALOR FROM [CCE_Reports].SECOPII.V_HistoricoContratos_Depurado V WHERE YEAR(V.AprovalDate) = {year} AND MONTH(V.AprovalDate) IN ({months}) AND V.ContractState NOT IN ('Borrador','Cancelado','En aprobación','enviado Proveedor')  AND [Código Entidad] = '702271321' GROUP BY ContractReference, V.[Nombre Entidad]) VALOR GROUP BY VALOR.[Nombre Entidad] ORDER BY SUM(VALOR.VALOR) DESC", cnxn_reports)

# Traducción de la consulta (Base bronce)
max_value_b = filtered_b.groupby(['ID_Contrato', 'Nombre_Entidad'])['Valor_contrato'].max().reset_index()
result_b = max_value_b.groupby('Nombre_Entidad').agg(CONTRATOS=('ID_Contrato', 'count'), VALOR=('Valor_contrato', 'sum')).reset_index()
result_b = result_b.sort_values(by='VALOR', ascending=False)
result_b = result_b.head(20)
result_b = result_b.rename(columns={'Nombre_Entidad': 'ENTIDAD', 'CONTRATOS': 'CONTRATOS', 'VALOR': 'VALOR'})

# Traducción de la consulta (Base silver)
max_value_s = filtered_s.groupby(['ID_Contrato', 'Nombre_Entidad'])['Valor_contrato'].max().reset_index()
result_s = max_value_s.groupby('Nombre_Entidad').agg(CONTRATOS=('ID_Contrato', 'count'), VALOR=('Valor_contrato', 'sum')).reset_index()
result_s = result_s.sort_values(by='VALOR', ascending=False)
result_s = result_s.head(20)
result_s = result_s.rename(columns={'Nombre_Entidad': 'ENTIDAD', 'CONTRATOS': 'CONTRATOS', 'VALOR': 'VALOR'})

test9[['ENTIDAD(B)', 'CONTRATOS(B)', 'VALOR(B)']] = result_b
test9[['ENTIDAD(S)', 'CONTRATOS(S)', 'VALOR(S)']] = result_s

try:
    RT9DC = int(test9['CONTRATOS'])    == int(test9['CONTRATOS(B)'])
    RT9DV = int(test9['VALOR'])        == int(test9['VALOR(B)'])
except:
    RT9DC = False
    RT9DV = False
    print('Error al validar el proceso de descarga')

try:
    RT9LC = int(test9['CONTRATOS(B)']) == int(test9['CONTRATOS(S)'])
    RT9LV = int(test9['VALOR(B)'])     == int(test9['VALOR(S)'])
except:
    RT9LC = False
    RT9LV = False
    print('Error al validar el proceso de limpieza')
    
print('Etapa de descarga:')
print('Contratos: ', RT9DC)
print('Valor: ',     RT9DV)
print('---')
print('Etapa de limpieza:')
print('Contratos: ', RT9LC)
print('Valor: ',     RT9LV)

Etapa de descarga:
Contratos:  True
Valor:  True
---
Etapa de limpieza:
Contratos:  True
Valor:  True


## Resultados

In [13]:
print('Test 1: ','Descarga:', RT1DC == True and RT1DV == True, 'Limpieza:', RT1LC == True and RT1LV == True)
print('Test 2: ','Descarga:', RT2DC == True and RT2DV == True, 'Limpieza:', RT2LC == True and RT2LV == True)
print('Test 3: ','Descarga:', RT3DC == True and RT3DV == True, 'Limpieza:', RT3LC == True and RT3LV == True)
print('Test 4: ','Descarga:', RT4DC == True and RT4DV == True, 'Limpieza:', RT4LC == True and RT4LV == True)
print('Test 5: ','Descarga:', RT5DC == True and RT5DV == True, 'Limpieza:', RT5LC == True and RT5LV == True)
print('Test 6: ','Descarga:', RT6DC == True and RT6DV == True, 'Limpieza:', RT6LC == True and RT6LV == True)
print('Test 7: ','Descarga:', RT7DC == True and RT7DV == True, 'Limpieza:', RT7LC == True and RT7LV == True)
print('Test 8: ','Descarga:', RT8DC == True and RT8DV == True, 'Limpieza:', RT8LC == True and RT8LV == True)
print('Test 9: ','Descarga:', RT9DC == True and RT9DV == True, 'Limpieza:', RT9LC == True and RT9LV == True)

Test 1:  Descarga: True Limpieza: True
Test 2:  Descarga: True Limpieza: True
Test 3:  Descarga: True Limpieza: False
Test 4:  Descarga: False Limpieza: True
Test 5:  Descarga: True Limpieza: True
Test 6:  Descarga: True Limpieza: True
Test 7:  Descarga: False Limpieza: True
Test 8:  Descarga: True Limpieza: True
Test 9:  Descarga: True Limpieza: True


In [14]:
test4[test4['RT4DV'] == False]

,MODALIDAD,CONTRATOS,VALOR,MODALIDAD(B),CONTRATOS(B),VALOR(B),MODALIDAD(S),CONTRATOS(S),VALOR(S),RT4DC,RT4DV,RT4LC,RT4LV
4,Compraventa,84,21221676939,Compraventa,84,21221676939,Compraventa,84,21221676939,True,False,True,True
6,DecreeLaw092/2017,4478,364411113427,DecreeLaw092/2017,4478,364411113427,DecreeLaw092/2017,4478,364411113427,True,False,True,True


In [15]:
test7

,MODALIDAD,CONTRATOS,VALOR,MODALIDAD(B),CONTRATOS(B),VALOR(B),MODALIDAD(S),CONTRATOS(S),VALOR(S),RT7DC,RT7DV,RT7LC,RT7LV
0,None,1,46998900,Contratación directa,1256,52248681191,contratacion directa,1256,52248681191,False,False,True,True
1,Contratación directa,1256,52248681191,Contratación régimen especial,1,200749195,contratacion regimen especial,1,200749195,False,False,True,True
2,Contratación régimen especial,1,200749195,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False


In [16]:
test4[test4['MODALIDAD'] == 'DecreeLaw092/2017']['VALOR'] - test4[test4['MODALIDAD'] == 'DecreeLaw092/2017']['VALOR(B)']

6   0
dtype: float64